In [17]:
# %pip install numpy pandas pickle
# %pip install matplotlib

In [18]:
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import pickle
import cv2

"""
- Conv
- ReLU
- MaxPool
- Conv
- MaxPool
- Flatten
- Dense
"""

'\n- Conv\n- ReLU\n- MaxPool\n- Conv\n- MaxPool\n- Flatten\n- Dense\n'

In [19]:
class ConvolutionLayer:

    def __str__(self) -> str:
        return "ConvolutionLayer"

    def __init__(self, n_filter, filter_size, stride, padding):
        self.filter_size = filter_size
        self.n_filter = n_filter
        self.stride = stride
        self.padding = padding

        self.filters = None
        self.biases = None

    
    def forward(self, input):        
        batch_size, n_channel, height, width = input.shape
        output_shape = (batch_size, self.n_filter, int((height - self.filter_size + 2*self.padding)/self.stride + 1), int((width - self.filter_size + 2*self.padding)/self.stride + 1))
        output = np.zeros(output_shape)

        if self.filters is None:
            self.filters = np.random.randn(self.n_filter, n_channel, self.filter_size, self.filter_size) / np.sqrt(2 / (self.filter_size * self.filter_size * n_channel))
        if self.biases is None:
            self.biases = np.random.randn(self.n_filter)

        if self.padding > 0:
            input = np.pad(input, ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)), 'constant')
        
        for b in range(batch_size):
            for c in range(self.n_filter):
                for h in range(height):
                    for w in range(width):
                        output[b, c, h, w] = np.sum(input[b, :, h*self.stride :h*self.stride + self.filter_size, w*self.stride : w*self.stride + self.filter_size] * self.filters[c, :, :, :]) + self.biases[c]

        return output


    def backward(self, output, learning_rate):
        # perform back propagation for convolution

        batch_size, n_channel, height, width = output.shape
        input_shape = (batch_size, n_channel, height, width)
        input = np.zeros(input_shape)

        if self.padding > 0:
            output = np.pad(output, ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)), 'constant')
        
        for b in range(batch_size):
            for c in range(self.n_filter):
                for h in range(height):
                    for w in range(width):
                        input[b, :, h*self.stride :h*self.stride + self.filter_size, w*self.stride : w*self.stride + self.filter_size] += output[b, c, h, w] * self.filters[c, :, :, :,]
                        #   did some numbo jumbo here (not sure if it's correct)
                        #   under this comment line
                        self.filters[c, :, :, :] -= learning_rate * output[b, c, h, w] * input[:, b, h*self.stride :h*self.stride + self.filter_size, w*self.stride : w*self.stride + self.filter_size]
                        self.biases[c] -= learning_rate * output[b, c, h, w]
        
        return input


In [20]:
class ReLUActivationLayer:

    def __str__(self) -> str:
        return "ReLUActivationLayer"

    def forward(self, input):
        return np.maximum(input, 0)

    def backward(self, output, learning_rate):
        return np.where(output > 0, 1, 0)


class MaxPoolingLayer:

    def __str__(self) -> str:
        return "MaxPoolingLayer"

    def __init__(self, pool_size, stride):
        self.pool_size = pool_size
        self.stride = stride

    def forward(self, input):
        batch_size, n_channel, height, width = input.shape

        output_h = int((height - self.pool_size)/self.stride + 1)
        output_w = int((width  - self.pool_size)/self.stride + 1)

        output_shape = (batch_size, n_channel, output_h, output_w)
        output = np.zeros(output_shape)

        for b in range(batch_size):
            for c in range(n_channel):
                for h in range(output_h):
                    for w in range(output_w):
                        output[b, c, h, w] = np.max(input[b, :, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size])

        return output

    def backward(self, output, learning_rate):
        batch_size, n_channel, height, width = output.shape
        input_shape = (batch_size, n_channel, height, width)
        input = np.zeros(input_shape)

        for b in range(batch_size):
            for c in range(n_channel):
                for h in range(height):
                    for w in range(width):
                        input[b, c, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size] = np.where(input[b, c, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size] == np.max(input[b, c, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size]), output[b, c, h, w], 0)

        return input    
        

class SoftMaxLayer:

    def __str__(self) -> str:
        return "SoftMaxLayer"

    def forward(self, input):
        val = input - np.max(input, axis=1, keepdims=True)
        val = np.exp(val) / np.exp(val).sum(axis=1, keepdims=True)
        return val

    def backward(self, output, learning_rate):
        return output


class FlatteningLayer:

    def __str__(self) -> str:
        return "FlatteningLayer"

    def forward(self, input):
        return input.reshape(input.shape[0], -1)

    def backward(self, output, learning_rate):
        return output.reshape(output.shape[0], -1)


class DenseLayer:

    def __init__(self, n_output):
        self.n_output = n_output
        self.weights = None
        self.biases = None

    def __str__(self) -> str:
        return "DenseLayer"

    def forward(self, input):

        batch_size, n_input = input.shape

        if self.weights is None:
            self.weights = np.random.randn(n_input, self.n_output) / np.sqrt(n_input)
        if self.biases is None:
            self.biases = np.random.randn(self.n_output)

        output = np.dot(input, self.weights) + self.biases
        return output


    def backward(self, output, learning_rate):
            
            batch_size, n_input = output.shape
    
            input = np.dot(output, self.weights.T)
            self.weights -= learning_rate * np.dot(output.T, input)
            self.biases -= learning_rate * output.sum(axis=0)
    
            return input
        



## Model Cell

In [21]:
class NeuralNetModel:

    def __init__(self) -> None:
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def __str__(self) -> str:
        return str(self.layers)
    
    def add(self, layer):
        self.layers.append(layer)
    
    def set_loss(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    
    def predict(self, x):
        out = x
        for layer in self.layers:
            out = layer.forward(out)
        return out
    
    def fit(self, x, y, learning_rate=0.01, epochs=1000):
        for i in range(epochs):
            out = self.predict(x)
            error = self.loss(y, out)
            if i % 100 == 0:
                print("Epoch %d, loss: %f" % (i, error))
            
            gradient = self.loss_prime(y, out)
            for layer in reversed(self.layers):
                gradient = layer.backward(gradient, learning_rate)
        
        print("Final loss: %f" % self.loss(y, self.predict(x)))
    
    def evaluate(self, x, y):
        out = self.predict(x)
        return self.loss(y, out)
    
    def accuracy(self, x, y):
        out = self.predict(x)
        return np.mean(np.argmax(out, axis=1) == np.argmax(y, axis=1))

In [22]:
def loss(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

def loss_prime(y_true, y_pred):
    return 2*(y_pred - y_true)

BASEDIR = "../../../numta"

def load_dataset():
    """Load the dataset from the base directory"""
    
    dataset = f"{BASEDIR}/training-a.csv"
    df = pd.read_csv(dataset)
    df = df[["filename", "digit"]]
    
    return df


def load_image(image_name):
    
    img = cv2.imread(f"{BASEDIR}/training-a/{image_name}")
    img = cv2.resize(img, (64, 64))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.invert(img)

    plt.imshow(img)
    plt.show()

    print(img)
    return img

# data = load_dataset()
# data.head

# X = data["filename"].values
# y = data["digit"].values

# load_image(X[0])

## Debug Cell

In [23]:
# batch_size, n_channel, height, width
# input_shape = (10, 10, 30, 30)
# input = np.random.randint(-2, 2, size=input_shape)
input = np.random.randn(10, 10, 32, 32)

# n_filter, filter_size, stride, padding
con = ConvolutionLayer(n_filter=5, filter_size=3, stride=1, padding=1)
relu = ReLUActivationLayer()
max = MaxPoolingLayer(pool_size=2, stride=1)
flat = FlatteningLayer()
dens = DenseLayer(n_output=10)
smax = SoftMaxLayer()

output = con.forward(input)
output = relu.forward(output)
output = max.forward(output)
output = flat.forward(output)
output = dens.forward(output)
output = smax.forward(output)

print(input.shape)
print(output.shape)

# print(input)
print(output)
print("Sum of final", output.sum())

(10, 10, 32, 32)
(10, 10)
[[5.99346376e-101 1.87739404e-124 9.90158452e-126 6.45105633e-112
  3.33333495e-011 1.12873676e-094 1.00000000e+000 4.08609673e-142
  4.05429668e-162 3.11545028e-037]
 [2.91443645e-096 4.21072153e-116 3.71093131e-118 4.82522920e-104
  9.86186626e-029 2.39800808e-082 4.76877275e-014 5.28253186e-096
  1.02425062e-129 1.00000000e+000]
 [5.96534074e-144 9.11312842e-166 4.87613127e-140 3.78100983e-131
  1.56766365e-021 4.89874148e-110 2.00556738e-031 8.46137031e-166
  3.01975523e-186 1.00000000e+000]
 [2.81634797e-116 8.36683806e-149 1.14117856e-149 3.10094048e-141
  3.79522708e-047 6.19782525e-107 1.00000000e+000 3.80151231e-157
  2.89700528e-203 2.93039976e-023]
 [5.34519646e-121 6.98746112e-110 2.78225887e-142 4.07312993e-097
  7.19876779e-032 5.96263073e-089 9.77600369e-016 2.17785554e-102
  1.15018511e-193 1.00000000e+000]
 [1.76987890e-124 3.23074242e-151 3.92116321e-135 1.09694713e-103
  3.34256692e-050 4.59648018e-104 1.21135605e-032 2.56265066e-137
  1.514